In [ ]:
import numpy as np
from scipy.optimize import least_squares

import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib import animation

from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display

%matplotlib inline

Załadowanie pliku z danymi i zapisanie jego linii do listy

In [ ]:
with open("python_zadanie_2.txt", "r") as task_file:
    task_file_lines = task_file.readlines()

Stworzenie słowników do wczytania i zapisania odpowiednich danych o anchorach:

In [ ]:
# number of blank lines in source file between title of data section and data
blank_lines_section = 6

# dictionary for anchors data
anchors_data = {'anchor_id': [], 'anchor_xy': []}
blank_count = 0

for line in task_file_lines[task_file_lines.index('1. Położenia anchorów\n') + blank_lines_section:]:
    if line != '\n':
        str_line = line.strip('\n').split(' ')
        num_line = [float(i) for i in str_line]
        num_line[0] = int(num_line[0])
        anchors_data['anchor_id'].append(num_line[0])
        anchors_data['anchor_xy'].append(tuple(num_line[1:3]))
        blank_count = 0
    else:
        blank_count += 1
    if blank_count > 1:
        break
        
print(anchors_data)

Stworzenie słowników do wczytania i zapisania odpowiednich danych o tagach:

In [ ]:
# dictionary for tags data
tags_data = {'tag_id': [], 'dist_to_anchors': [], 'tag_xy': []}

blank_count = 0
for line in task_file_lines[task_file_lines.index('2. Pomiary odległości obiektów od anchorów\n') + blank_lines_section:]:
    if line != '\n':
        str_line = line.strip('\n').split(' ')
        num_line = [float(i) for i in str_line]
        num_line[0] = int(num_line[0])
        if num_line[0] not in tags_data['tag_id']:
            tags_data['tag_id'].append(num_line[0])
            tags_data['dist_to_anchors'].append([num_line[1:]])
        else:
            index = tags_data['tag_id'].index(num_line[0])
            tags_data['dist_to_anchors'][index].append(num_line[1:])

        blank_count = 0
    else:
        blank_count += 1
    if blank_count > 1:
        break
print("ID tagów zgodnie z kolejnością w zapisie") 
print(tags_data['tag_id'])

Posortowanie tagow wzgledem ich id

In [ ]:
# sort tags_data by tag_id
tags_data['tag_id'], tags_data['dist_to_anchors'] = zip(*sorted(zip(tags_data['tag_id'], tags_data['dist_to_anchors'])))

tags_data['tag_id'] = list(tags_data['tag_id'])
tags_data['dist_to_anchors'] = list(tags_data['dist_to_anchors'])

print("ID tagów zgodnie z kolejnością w zapisie") 
print(tags_data['tag_id'])

Stworzenie słowników do wczytania i zapisania odpowiednich danych o wielokątach:

In [ ]:
# dictionary for polygons data
polys_data = {'poly_id': [], 'nodes_id': [], 'contained_tags_ids': [], 'contained_tags_spec': []}

blank_count = 0

for line in task_file_lines[task_file_lines.index('3. Opis wielokątów\n') + blank_lines_section:]:
    if line != '\n':
        str_line = line.strip('\n').replace('[', '').replace(']', '').split(' ')
        num_line = [int(i) for i in str_line]
        polys_data['poly_id'].append(num_line[0])
        polys_data['nodes_id'].append(num_line[1:])
        blank_count = 0
    else:
        blank_count += 1
    if blank_count > 1:
        break

print(polys_data)

Stworzenie słowników do wczytania i zapisania odpowiednich danych o wierzchołkach wielokątów:

In [ ]:
# dictionary for polygons' nodes data
nodes_data = {'node_id': [], 'node_xy': []}

blank_count = 0

for line in task_file_lines[task_file_lines.index('4. Położenia wierzchołków wielokątów\n') + blank_lines_section:]:
    if line != '\n':
        str_line = line.strip('\n').split(' ')
        num_line = [float(i) for i in str_line]
        num_line[0] = int(num_line[0])
        nodes_data['node_id'].append(num_line[0])
        nodes_data['node_xy'].append(tuple(num_line[1:3]))
        blank_count = 0
    else:
        blank_count += 1
    if blank_count > 1:
        break

print(nodes_data)

# Obliczenie położenia tagów na podstawie ich odległości od anchorów

Obliczenia te zrealizujemy tworząc uklad czterech równań nieliniowych- (równań okręgów o środku w danym anchorze i promieniu równym odległości anchora od taga) - o dwoch niewiadomych (współrzędnych x, y taga). Układ ten będzie nadokreślony (więcej równań niż niewiadomych) - nie będzie miał jednego jednoznacznego rozwiązania. Aby uzyskac satysfakcjonujące - optymalne rozwiązanie poslużymy się algorytmem Levenberga-Marquardta (służącego do optymalizacji nieliniowej). Użyjemy go stosując funkcję *least_squares* z modułu *scipy.optimize*.

In [ ]:
def equations(X_vec, **kwargs):
    """
    Calculate all equations for equations system to solve
    
    Parameters:
        X_vec: vector of arguments (solutions)
        **kwargs: anchors positions - [(x0, y0),..., (xn, yn)] and distances to anchors [d0, d1, d2, d3]
    Returns:
        F: vector of calculated values of equations for given X_vec arguments
    """
    
    F = []
    for i in range(4):
        F.append((kwargs['anchors'][i][0] - X_vec[0]) ** 2 + (kwargs['anchors'][i][1] - X_vec[1]) ** 2
                 - kwargs['distances'][i] ** 2)
    return np.array(F)

def getTagPosition(anchors, distances):
    """Calculate tag position using least_squares method (Levenberg-Marquardt algorithm)"""
    
    # define initial solution approximation which will be beginning of algorithm iterations
    X_vec_0 = np.array([0, 0])
    tag_data = {'anchors': anchors, 'distances': distances}
    result = least_squares(equations, X_vec_0, method='lm', kwargs=tag_data)
    return tuple(result.x)

Przeprowadzenie obliczeń i zapisanie współrzędnych wszystkich tagów i ich punktów

In [ ]:
for counter1, tag_distances in enumerate(tags_data['dist_to_anchors']):
    for counter2, tag_dist in enumerate(tag_distances):
        if counter2 == 0:
            tags_data['tag_xy'].append([getTagPosition(anchors_data['anchor_xy'], tag_dist)])
        else:
            tags_data['tag_xy'][counter1].append(getTagPosition(anchors_data['anchor_xy'], tag_dist))

In [ ]:
# limits of x, y axes used to print future plots
plot_limits = [-1.25, 1.25]
plot_size = [12, 12]

def displayTagPositionCalc(tag_id, point_num):
    fig, ax = plt.subplots()
    ax.figure.set_size_inches(*plot_size)
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    
    if tag_id in tags_data['tag_id'] and point_num < len(tags_data['tag_xy'][tags_data['tag_id'].index(tag_id)]):
        plt.scatter(*tags_data['tag_xy'][tags_data['tag_id'].index(tag_id)][point_num], s=40)
        for i in range(len(anchors_data['anchor_id'])):
            print("Odległość od {}. anchora: {}".format(i, tags_data['dist_to_anchors']
                                                    [tags_data['tag_id'].index(tag_id)][point_num][i]))
            ax.add_artist(plt.Circle(anchors_data['anchor_xy'][i],
                                     tags_data['dist_to_anchors'][tags_data['tag_id'].index(tag_id)][point_num][i],
                                     fill=False, color='r'))
        print("\nPołożenie {}_{} taga: {}".format(tag_id, point_num, tags_data['tag_xy'][tags_data['tag_id'].index(tag_id)][point_num]))
    plt.show()
    
    
disp_tag_posit_calc = interactive(displayTagPositionCalc,
                                  tag_id=widgets.IntSlider(min=0,max=max(tags_data['tag_id']),step=1,value=0),
                                  point_num=widgets.IntSlider(min=0,max=max(tags_data['tag_id']),step=1,value=0))
display(disp_tag_posit_calc)

In [ ]:



def displayTag(tag_id):       
    fig, ax = plt.subplots()
    ax.figure.set_size_inches(*plot_size)
    ax.set_xlim(*plot_limits)
    ax.set_ylim(*plot_limits)
    if tag_id == -1:
        ax.scatter(*zip(*[j for i in tags_data['tag_xy'] for j in i]))
        # for tag_counter, tag_points in enumerate(tags_data['tag_xy']):
        #     for tag_points_counter, tag_point in enumerate(tag_points):
        #         print(tag_points_counter)
        #         annotate_id = "tag{}_{}".format(tag_counter, tag_points_counter)
        #         plt.annotate(annotate_id, tag_point)
    elif tag_id in tags_data['tag_id']:
        ax.scatter(*zip(*(tags_data['tag_xy'][tags_data['tag_id'].index(tag_id)])))
        # for tag_points_counter, tag_point in enumerate(tags_data['tag_xy'][tags_data['tag_id'].index(tag_id)]):
        #     annotate_id = "tag{}-{}".format(tag_id, tag_points_counter)
        #     plt.annotate(annotate_id, tag_point)
        
    plt.show()
    
    if tag_id == -1:
        for i in tags_data['tag_xy']:
            print(i)
    elif tag_id in tags_data['tag_id']:
        for i in tags_data['tag_xy'][tags_data['tag_id'].index(tag_id)]:
            print(i)

    
disp_tag_id = interactive(displayTag, tag_id=widgets.IntSlider(min=-1,max=max(tags_data['tag_id']),step=1,value=0))
display(disp_tag_id)

# Zbadanie czy dane tagi należą do danych wielokątów

In [ ]:
def getLineEquationCoeffs(seg):
    x1, y1 = seg[0][0], seg[0][1]
    x2, y2 = seg[1][0], seg[1][1]

    A = y2 - y1
    B = x1 - x2
    C = (x2 * y1) - (x1 * y2)

    return A, B, C

def doSegmentsIntersect(segA, segB):
    xA_1, yA_1 = segA[0][0], segA[0][1]
    xA_2, yA_2 = segA[1][0], segA[1][1]

    xB_1, yB_1 = segB[0][0], segB[0][1]
    xB_2, yB_2 = segB[1][0], segB[1][1]

    # seg1 as inf line
    # equation of line1 as Ax + By + C = 0

    a1, b1, c1 = getLineEquationCoeffs(segA)

    # check equation for values of seg2 coords
    dB_1 = a1 * xB_1 + b1 * yB_1 + c1
    dB_2 = a1 * xB_2 + b1 * yB_2 + c1

    #if both seg2 points are on the same side of line1 - no intersection
    if dB_1 * dB_2 > 0:
        return False

    a2, b2, c2 = getLineEquationCoeffs(segB)

    dA_1 = a2 * xA_1 + b2 * yA_1 + c2
    dA_2 = a2 * xA_2 + b2 * yA_2 + c2

    if dA_1 * dA_2 > 0:
        return False

    # if segments are collinear
    if (a1 * b2) - (a2 * b1) == 0.:
        return False

    return True

In [ ]:
def isTagInPoligon(tag_xy, poly_xy):

    poly_min_x, poly_max_x = min([pnt[0] for pnt in poly_xy]), max([pnt[0] for pnt in poly_xy])
    poly_min_y, poly_max_y = min([pnt[1] for pnt in poly_xy]), max([pnt[1] for pnt in poly_xy])

    # polygon bounding box check
    if tag_xy[0] > poly_max_x or tag_xy[0] < poly_min_x:
        return False
    if tag_xy[1] > poly_max_y or tag_xy[1] < poly_min_y:
        return False

    # create reference point which together with tag will create segment to check intersections
    eps = 0.2
    refPoint = (poly_max_x + eps * abs(poly_max_x), poly_max_y + eps * abs(poly_max_y))
    refPoints = [refPoint, (refPoint[0] + eps / 4, refPoint[1]), (refPoint[0], refPoint[1] - eps / 4),
                 (refPoint[0] - eps / 4, refPoint[1]), (refPoint[0], refPoint[1] + eps / 4)]

    # check for all reference points to make sure collinearity doesnt spoil result
    for refPnt in refPoints:
        tag_seg = [tag_xy, refPnt]
        intersects_count = 0
        for counter, poly_nod in enumerate(poly_xy):
            poly_seg = [poly_nod, poly_xy[counter - 1]]

            if doSegmentsIntersect(tag_seg, poly_seg):
                intersects_count += 1

        if intersects_count % 2 == 1:
            return True
        else:
            continue
    return False

Zapisanie które tagi oraz które ich punkty należały do danego wielokąta

In [ ]:
for poly_nodes in polys_data['nodes_id']:
    tags_poly_contained_spec = {'tag_id': tags_data['tag_id'], 'contained_state': []}
    tags_contained_spec = []

    # create list of points of current polygon
    poly_xy = []
    for node_id in poly_nodes:
        poly_xy.append(nodes_data['node_xy'][nodes_data['node_id'].index(node_id)])
    tags_contained_ids = []
    for tags_multi in tags_data['tag_xy']:
        tags_contained_spec_multi = []
        for tag in tags_multi:
            if isTagInPoligon(tag, poly_xy):
                tags_contained_spec_multi.append(True)
            else:
                tags_contained_spec_multi.append(False)

        tags_poly_contained_spec['contained_state'].append(tags_contained_spec_multi)
        if True in tags_contained_spec_multi:
            tags_contained_ids.append(tags_data['tag_id'][tags_data['tag_xy'].index(tags_multi)])

    polys_data['contained_tags_spec'].append(tags_poly_contained_spec)
    polys_data['contained_tags_ids'].append(tags_contained_ids)

print('contained_tags_spec')
print(polys_data['contained_tags_spec'])

Dodanie do danych o wielokątach list punktów potrzebnych do ich rysowania

In [ ]:
polys_lines = []
for poly_nodes in polys_data['nodes_id']:
    poly_lines = []

    for counter1, node_id in enumerate(poly_nodes):
        point1 = (nodes_data['node_xy'][nodes_data['node_id'].index(node_id)])
        point2 = (nodes_data['node_xy'][nodes_data['node_id'].index(poly_nodes[counter1 - 1])])
        poly_line = [point1, point2]
        poly_lines.append(poly_line)
    polys_lines.append(poly_lines)
polys_data['poly_lines'] = polys_lines

Przedstawienie wielokątów

In [ ]:
lc0 = LineCollection(polys_data['poly_lines'][0], colors='r')
lc1 = LineCollection(polys_data['poly_lines'][1], colors='g', linewidths=2)
lc2 = LineCollection(polys_data['poly_lines'][2], colors='b')
lc3 = LineCollection(polys_data['poly_lines'][3], colors='y')
lc4 = LineCollection(polys_data['poly_lines'][4], colors='m')
fig, ax = plt.subplots()
ax.set_xlim(*plot_limits)
ax.set_ylim(*plot_limits)
ax.add_collection(lc0); ax.add_collection(lc1); ax.add_collection(lc2); ax.add_collection(lc3); ax.add_collection(lc4)
plt.show()

Przedstawienie tagów:

In [ ]:
def displayTagAndPolys(tag_id):       
    lc0 = LineCollection(polys_data['poly_lines'][0], colors='r')
    lc1 = LineCollection(polys_data['poly_lines'][1], colors='g', linewidths=2)
    lc2 = LineCollection(polys_data['poly_lines'][2], colors='b')
    lc3 = LineCollection(polys_data['poly_lines'][3], colors='y')
    lc4 = LineCollection(polys_data['poly_lines'][4], colors='m')
    fig, ax = plt.subplots()
    ax.figure.set_size_inches(*plot_size)
    ax.set_xlim(*plot_limits)
    ax.set_ylim(*plot_limits)
    ax.add_collection(lc0); ax.add_collection(lc1); ax.add_collection(lc2); ax.add_collection(lc3); ax.add_collection(lc4)
    if tag_id == -1:
        ax.scatter(*zip(*[j for i in tags_data['tag_xy'] for j in i]))
    elif tag_id in tags_data['tag_id']:
        ax.scatter(*zip(*(tags_data['tag_xy'][tags_data['tag_id'].index(tag_id)])))
    plt.show()
    
disp_tagpolys_id = interactive(displayTagAndPolys, tag_id=widgets.IntSlider(min=-1,max=max(tags_data['tag_id']),step=1,value=0))
display(disp_tagpolys_id)

# Występowanie tagów w wielokątach

Wyrysowanie oraz wypisanie tagów zawartych w poszczególnych wielokątach. <br>
(wybór -1 odpowiada sytuacji wielokąta 1. z wyłączeniem pola 2.)

In [ ]:
def printTagsContainedInPoly(poly_id):
    
    fig, ax = plt.subplots()
    ax.figure.set_size_inches(*plot_size)
    ax.set_xlim(*plot_limits)
    ax.set_ylim(*plot_limits)
    contained_points = []
    contained_tags_ids = []
    
    # when we inspect area of 1 excluding 2
    if poly_id == -1:
        print("Tagi należące do wielokąta 1. z wyłączeniem obszaru 2.:")
        lc1 = LineCollection(polys_data['poly_lines'][1], colors='r')
        lc2 = LineCollection(polys_data['poly_lines'][2], colors='b')
        ax.add_collection(lc1)
        ax.add_collection(lc2)
        
        for tag_counter, tag_spec in enumerate(polys_data['contained_tags_spec'][1]['contained_state']):
            for occur_counter, tag_state in enumerate(tag_spec):
                if tag_state is True and \
                        polys_data['contained_tags_spec'][2]['contained_state'][tag_counter][occur_counter] is False:
                    contained_points.append(tags_data['tag_xy'][tag_counter][occur_counter])
                    if polys_data['contained_tags_spec'][1]['tag_id'][tag_counter] not in contained_tags_ids:
                        contained_tags_ids.append(polys_data['contained_tags_spec'][1]['tag_id'][tag_counter])
        
    # normally
    else:
        print("Tagi należące do wielokąta {}.:".format(poly_id))
        lc = LineCollection(polys_data['poly_lines'][poly_id], colors='r')
        ax.add_collection(lc)
        for tag_counter, tag_spec in enumerate(polys_data['contained_tags_spec'][poly_id]['contained_state']):
            for occur_counter, tag_state in enumerate(tag_spec):
                if tag_state == True:
                    contained_points.append(tags_data['tag_xy'][tag_counter][occur_counter])
                    if polys_data['contained_tags_spec'][poly_id]['tag_id'][tag_counter] not in contained_tags_ids:
                        contained_tags_ids.append(polys_data['contained_tags_spec'][poly_id]['tag_id'][tag_counter])
                        
    print(contained_tags_ids)
    ax.scatter(*zip(*contained_points))

print_tags_contained_in_poly = interactive(printTagsContainedInPoly, poly_id=widgets.IntSlider(min=-1,max=max(polys_data['poly_id']),step=1,value=0))
display(print_tags_contained_in_poly)

Zapisanie danych o szczególnej ilości wystąpień poszczególnych tagów w wielokątach (dla każdego ich punktu)

In [ ]:
tags_data['tag_occurance_spec'] = [[0]] * len(tags_data['tag_id'])
for counter, occurs in enumerate(tags_data['tag_xy']):
    tags_data['tag_occurance_spec'][counter] = [0] * len(occurs)


for poly in polys_data['contained_tags_spec']:
    for tag_counter, tags_spec in enumerate(poly['contained_state']):
        for occur_counter, tag_spec in enumerate(tags_spec):
            if tag_spec == True:
                tags_data['tag_occurance_spec'][tag_counter][occur_counter] += 1
print(tags_data['tag_occurance_spec'])

Zapisanie danych o ogólnej ilości wystąpień poszczególnych tagów

In [ ]:
tags_data['tag_occurances_sum'] = [0] * len(tags_data['tag_id'])
for tag_counter, tag_occurs in enumerate(tags_data['tag_occurance_spec']):
    for occur in tag_occurs:
        tags_data['tag_occurances_sum'][tag_counter] += occur
            
print(tags_data['tag_occurance_spec'])

Wyznaczenie najczęściej występujących tagów zawartych w wielokątach

In [ ]:
most_occur_tags = [i for count, i in enumerate(tags_data['tag_id'])
                   if tags_data['tag_occurances_sum'][count] == max(tags_data['tag_occurances_sum'])]
print("Tago z największą ilością wystąpień w wielokątach (tag_id): {}".format(most_occur_tags))

Obliczenie ilości występowania punktów i tagów nie zawartych w żadnym wielokącie

In [ ]:
blank_tags = {'blank_points_num': 0, 'blank_tags_num': 0, 'fully_blank_tags_num': 0}
for tag_counter, tag_occurs in enumerate(tags_data['tag_occurance_spec']):
    blank_tag_check_state = False
    for occur in tag_occurs:
        if occur == 0:
            blank_tags['blank_points_num'] += 1
            if blank_tag_check_state == False:
                blank_tags['blank_tags_num'] += 1
                blank_tag_check_state = True
blank_tags['fully_blank_tags_num'] = tags_data['tag_occurances_sum'].count(0)
print("{}\tilość punktów nie zawarta w żadnym wielokącie".format(blank_tags['blank_points_num']))
print("{}\tilość tagów nie zawarta w żadnym wielokącie co najmniej raz".format(blank_tags['blank_tags_num']))
print("{}\tilość tagów nigdy nie zawarta w wielokącie".format(blank_tags['fully_blank_tags_num']))